<a href="https://colab.research.google.com/github/MalihaUCF/Machine-Learning-Course-Assignments--Spring-2019/blob/master/Assignment1/Problem5/Problem5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# Feature Engineering 
# Extending my solution to problem 4
# Loading MNIST 

from keras.datasets import mnist
from keras import layers
from keras import models
import numpy as np
from keras.utils import to_categorical


(train_images_original,train_labels_original),(test_images_original,test_labels_original)=mnist.load_data()  

print('Training Data Shape',train_images_original.shape)
print('Test Data Shape',test_images_original.shape)
print('The train and test labels look like this,' ,train_labels_original.shape,test_labels_original.shape)

train_images = train_images_original.reshape(60000, 28*28)  #reshaped
test_images = test_images_original.reshape(10000, 28*28) 
print(train_images.shape)
print(test_images.shape)



Training Data Shape (60000, 28, 28)
Test Data Shape (10000, 28, 28)
The train and test labels look like this, (60000,) (10000,)
(60000, 784)
(10000, 784)


In [0]:


#This function here converts the images to binary - Black and White images
def binary_images(train_images):

  train_images_binary=np.empty(shape=(train_images.shape))
  print(train_images_binary.shape)

  for x in range(0,train_images.shape[0]):
    image_array=train_images[x]
    #print(image_array)
    binary=np.where(image_array > 0,1,0)
    #print(binary)

    train_images_binary[x]=binary
    
  return train_images_binary
 

#This function adds an extra feature to the binary images
#The extra feature is the ratio of number of white pixels to black pixels , later we will add number of white regions in image
def number_BW_ratios(train_images):
  
  train_images_new=np.empty(shape=(train_images.shape[0],785))  #defines new empty array with new feature 
  
  
  for index in range(0,train_images.shape[0]):
    image=train_images[index]
    
    #loop for counting black and white pixels
    #then calculating ratios
    white_pixels=0
    black_pixels=0
    for pixel in range(0,784):
      if (image[pixel]==0):
        white_pixels+=1
        
      else:
        black_pixels+=1
    #print('White',white_pixels)   
    ratio_BW=(white_pixels/black_pixels)#.astype('float32') 
    #print(ratio_BW)
        
    extra_feature=ratio_BW
    extra_feature/=255
    image_new=np.empty(shape=785)
    
    for x in range(0,784):
      image_new[x]=image[x]

    image_new[784]=extra_feature
    train_images_new[index]=image_new
    
  return train_images_new


#Here we convert images to binary, then calculate the ratio of BW pixels before concatenating with original feature vector
train_images_binary=binary_images(train_images)   #send reshaped images here 
test_images_binary=binary_images(test_images)
print('Binary Images',train_images_binary.shape)
print(train_images[0][27])

#The set of concatenated features images with ratios which will sent to the network for training

train_images_new2=number_BW_ratios(train_images_binary)  
test_images_new2=number_BW_ratios(test_images_binary)
print(test_images_new2.shape)

 


(60000, 784)
(10000, 784)
Binary Images (60000, 784)
0
(10000, 785)


In [0]:

#Defining the Single layered Network

model=models.Sequential()
model.add(layers.Dense((28*28),activation='relu',input_shape=((28*28)+1,)))
model.add(layers.Dense(10,activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 784)               616224    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                7850      
Total params: 624,074
Trainable params: 624,074
Non-trainable params: 0
_________________________________________________________________


In [0]:
train_labels = to_categorical(train_labels_original,10)
test_labels = to_categorical(test_labels_original,10)



#Compiling the network with Black and white ratio as extra feature

epochs=20
batch_size=128
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_images_new2, train_labels, epochs=epochs,batch_size=batch_size, validation_data=(test_images_new2, test_labels))





Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s 147us/step - loss: 0.8898 - acc: 0.7971 - val_loss: 0.4873 - val_acc: 0.8809
Epoch 2/20
60000/60000 [==============================] - 8s 141us/step - loss: 0.4384 - acc: 0.8838 - val_loss: 0.3781 - val_acc: 0.8976
Epoch 3/20
60000/60000 [==============================] - 9s 146us/step - loss: 0.3694 - acc: 0.8977 - val_loss: 0.3376 - val_acc: 0.9075
Epoch 4/20
60000/60000 [==============================] - 9s 148us/step - loss: 0.3353 - acc: 0.9052 - val_loss: 0.3108 - val_acc: 0.9125
Epoch 5/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.3127 - acc: 0.9110 - val_loss: 0.2953 - val_acc: 0.9166
Epoch 6/20
60000/60000 [==============================] - 8s 141us/step - loss: 0.2957 - acc: 0.9156 - val_loss: 0.2797 - val_acc: 0.9209
Epoch 7/20
60000/60000 [==============================] - 8s 141us/step - loss: 0.2818 - acc: 0.9193 - val_loss: 0.2675 - 

In [0]:

#In this part of the code, we will use Connected components algorithm and then classify using 
#number of white regions per image

# A class is defined that will take the image and calculate the connected components using DFS

class Connected_components: 
  
    def __init__(self,image, height, width): 
        self.graph = image
        self.height = height 
        self.width = width 
        
  
    def visited(self, i, j, visited): 
        return (i >= 0 and i < self.height and 
                j >= 0 and j < self.width and 
                not visited[i][j] and self.graph[i][j]) 
              

    def DFS(self, i, j, visited): 
        rowNbr = [-1, -1, -1,  0, 0,  1, 1, 1]; 
        colNbr = [-1,  0,  1, -1, 1, -1, 0, 1];   
        visited[i][j] = True
  
        #we use 8 neighborhood 
        for k in range(8): 
            if self.visited(i + rowNbr[k], j + colNbr[k], visited): 
                self.DFS(i + rowNbr[k], j + colNbr[k], visited) 
  

    def count_white_regions(self): 
        visited = [[False for j in range(self.width)]for i in range(self.height)] 

        count = 0
        for i in range(self.height): 
            for j in range(self.width): 
                if visited[i][j] == False and self.graph[i][j] ==1: 
                    self.DFS(i, j, visited) 
                    count += 1
  
        return count



#get binary images with shape (..,28,28)
train_images_binary=binary_images(train_images_original)  
test_images_binary=binary_images(test_images_original)


#define function that uses the Connected components class above and calculates number of white regions
def connected_components(train_images):
  
  train_images_new=np.empty(shape=(train_images.shape[0],785))  #defines new vector of shape (60000,785) or (10000,785)
  
  for index in range(0,train_images.shape[0]):
    
    image=train_images[index]
    num_white_regions=Connected_components(image,28, 28).count_white_regions()
    extra_feature=num_white_regions
    image_new=np.empty(shape=785)
    image=train_images[index].reshape(28*28)
    
    for x in range(0,784):
      image_new[x]=image[x]

    image_new[784]=extra_feature
    train_images_new[index]=image_new
    
  return train_images_new


#This function here adds the extra feature -number of white regions to the original image feature vector 
def add_feature(train_images):
  
  train_images_new=np.empty(shape=(train_images.shape[0],785))  #defines new vector of shape (60000,785) or (10000,785)
  print('new',train_images_new.shape)
  white_region_dict={0:2, 1:1,2:1,3:1,4:1,5:1,6:2,7:1,8:3,9:2}
  
  for index in range(0,train_images.shape[0]):
    image=train_images[index]
    image_label=train_labels[index]
    #print(image_label)
    extra_feature=white_region_dict[image_label]
    #print(extra_feature)
    image_new=np.empty(shape=785)
    
    for x in range(0,784):
      image_new[x]=image[x]

    image_new[784]=extra_feature
    #print(image_new)
    #print(image_new.shape)
    train_images_new[index]=image_new
    
  return train_images_new


#get binary images again
train_images_binary1=binary_images(train_images_original)   #send original images here 
test_images_binary1=binary_images(test_images_original)
print('Binary Images',train_images_binary1.shape)
print(train_images[0][27])


#here we find number of white regions and concatenate

train_images_new1=connected_components(train_images_binary1) 
print(train_images_new1.shape)
print(train_images_new1[59000].shape)


test_images_new1=connected_components(test_images_binary1) 
print(test_images_new1.shape)
print(test_images_new1[1000].shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 28, 28)
(10000, 28, 28)
Binary Images (60000, 28, 28)
0
(60000, 785)
(785,)
(10000, 785)
(785,)


In [0]:
# Here we train our classifier using Number of white regions as extra features

train_labels1 = to_categorical(train_labels_original,10)
test_labels1 = to_categorical(test_labels_original,10)

epochs=20
batch_size=128
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_images_new1, train_labels1, epochs=epochs,batch_size=batch_size, validation_data=(test_images_new1, test_labels1))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s 156us/step - loss: 0.1826 - acc: 0.9481 - val_loss: 0.1854 - val_acc: 0.9451
Epoch 2/20
60000/60000 [==============================] - 9s 145us/step - loss: 0.1783 - acc: 0.9494 - val_loss: 0.1811 - val_acc: 0.9472
Epoch 3/20
60000/60000 [==============================] - 9s 146us/step - loss: 0.1743 - acc: 0.9504 - val_loss: 0.1781 - val_acc: 0.9481
Epoch 4/20
60000/60000 [==============================] - 9s 148us/step - loss: 0.1704 - acc: 0.9516 - val_loss: 0.1741 - val_acc: 0.9486
Epoch 5/20
60000/60000 [==============================] - 9s 145us/step - loss: 0.1667 - acc: 0.9524 - val_loss: 0.1711 - val_acc: 0.9501
Epoch 6/20
60000/60000 [==============================] - 9s 143us/step - loss: 0.1632 - acc: 0.9533 - val_loss: 0.1681 - val_acc: 0.9509
Epoch 7/20
60000/60000 [==============================] - 10s 158us/step - loss: 0.1598 - acc: 0.9547 - val_loss: 0.1659 -